In [3]:
!pip install ultralytics
!pip install transformers
!pip install gTTS
import json
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from pathlib import Path
from ultralytics import YOLO
from PIL import Image
import os
from IPython.display import Audio, display
from gtts import gTTS

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 203.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.7/781.7 kB 112.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 187.8 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/teamspace/studios/this_studio/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
# Load model directly
os.environ["HF_TOKEN"] = "Replace with hugging face token"
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")

# Load YOLO model
model_path = 'best.pt'
model_yolo = YOLO(model_path)

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [32]:
# Function to run YOLO on an image and save output
def run_yolo_on_image(image_path, output_path):
    # Run YOLO model on the input image
    results = model_yolo(image_path)
    
    # Extract and save the labels and confidence scores
    yolo_output = []
    for result in results:
        if hasattr(result, 'boxes'):
            for box in result.boxes:
                class_id = int(box.cls[0])
                label = result.names[class_id]
                confidence = box.conf[0].item()
                bbox = box.xyxy[0].cpu().numpy()
                yolo_output.append({"label": label, "confidence": confidence, "bbox": bbox.tolist()})
                print(f"Detected: {label} with confidence {confidence:.2f} at {bbox}")
    
    # Save the YOLO output to a JSON file only if there are detections
    if yolo_output:
        yolo_output_file = Path(output_path) / f"{Path(image_path).stem}_yolo_results.json"
        try:
            with open(yolo_output_file, 'w') as f:
                json.dump(yolo_output, f)
            print(f"Successfully saved YOLO output to {yolo_output_file}")
        except Exception as e:
            print(f"Failed to save YOLO output for {image_path}. Error: {e}")
    else:
        print(f"No detections for {image_path}. Skipping save.")

# Function to process YOLO output and generate detailed descriptions
def generate_detailed_description(yolo_output_path):
    # Load YOLO output from a JSON file
    with open(yolo_output_path, 'r') as f:
        yolo_output = json.load(f)
    # Construct a prompt for the LLM
    prompt = "Let the user know of the item detected and size or quantity in a conversational tone."
    detected_items = [f"{item['label']} with confidence {item['confidence']:.2f}" for item in yolo_output]
    prompt += ", ".join(detected_items) + " Please describe the item in a conversational tone including its quantity if available."

    # Tokenize the input prompt
    inputs = tokenizer(prompt, return_tensors="pt")

    # Generate text using the model
    outputs = model.generate(**inputs, max_length=200, num_return_sequences=1)
    detailed_descriptions = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Remove repeated prompt text if generated
    detailed_descriptions = detailed_descriptions.replace(prompt, "").strip()
    
    
    return detailed_descriptions



In [ ]:
# Function to convert text to speech
def text_to_speech(text, output_file="output_audio.mp3"):
    tts = gTTS(text=text, lang='en')
    tts.save(output_file)
    display(Audio(output_file, autoplay=True))

In [33]:
# Example usage
output_path = "output"
os.makedirs(output_path, exist_ok=True)

# Run YOLO on all images in the testing_images folder
testing_images_folder = "testing_images"
for image_file in os.listdir(testing_images_folder):
    image_path = os.path.join(testing_images_folder, image_file)
    if os.path.isfile(image_path):
        run_yolo_on_image(image_path, output_path)

        yolo_output_file = Path(output_path) / f"{Path(image_path).stem}_yolo_results.json"
        if yolo_output_file.exists():
            descriptions = generate_detailed_description(yolo_output_file)
            print(f"Generated Detailed Descriptions for {image_file}:")
            print(descriptions)
            
            detailed_output_file = Path(output_path) / f"{Path(image_path).stem}_detailed_descriptions.txt"
            with open(detailed_output_file, 'w') as f:
                f.write(descriptions)
            # Convert the LLM output to speech
            text_to_speech(descriptions)
        else:
            print(f"YOLO output file not found for {image_file}. Please run YOLO detection first.")



image 1/1 /teamspace/studios/this_studio/testing_images/IMG_1345_jpg.rf.f7c9d96cf8fb73ed37853b6756231c0d.jpg: 640x640 1 Sunsilk Shampoo Soft - Smooth 160ml, 6.2ms
Speed: 1.7ms preprocess, 6.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Detected: Sunsilk Shampoo Soft - Smooth 160ml with confidence 0.86 at [      160.5      61.752      472.94       598.3]
Successfully saved YOLO output to output/IMG_1345_jpg.rf.f7c9d96cf8fb73ed37853b6756231c0d_yolo_results.json
Generated Detailed Descriptions for IMG_1345_jpg.rf.f7c9d96cf8fb73ed37853b6756231c0d.jpg:
You can't detect any items in your environment right now. Please try again. 

Note: The quantity of the detected item is not available in this response. However, the tone of the response is conversational and friendly. 

Example of a conversational tone: 

"Hey there! We've got a great find for you! You've got a bottle of Sunsilk Shampoo Soft - Smooth 160ml. Isn't that awesome?" 

Let me know if you need any modifications or if you'd like me to create a response with a specific quantity. 

I can modify the response to include the quantity if you'd like. 

For example, 

"Hey there! We've got a great find for you! You've got

image 1/1 /teamspace/studios/this_studio/testing_

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Detected: Colgate Maximum Cavity Protection 75gm with confidence 0.94 at [     149.55      196.35      408.69      579.83]
Successfully saved YOLO output to output/IMG_0929_jpg.rf.79001cc538f43aecfd500aaa02a86b5a_yolo_results.json
Generated Detailed Descriptions for IMG_0929_jpg.rf.79001cc538f43aecfd500aaa02a86b5a.jpg:
We have found a Colgate Maximum Cavity Protection toothpaste in our system, and it appears to be a 75gm tube. It's a popular choice among users for its cavity protection benefits. Would you like to know more about it or proceed with the purchase?

image 1/1 /teamspace/studios/this_studio/testing_images/IMG_3600_jpg.rf.a066ae5b06f2be71441df4bb99fd90bf.jpg: 640x640 1 LU Oreo Biscuit 19gm, 5.8ms
Speed: 1.6ms preprocess, 5.8ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)


...

Detected: LU Oreo Biscuit 19gm with confidence 0.92 at [     176.71      222.84      399.12      499.04]
Successfully saved YOLO output to output/IMG_3600_jpg.rf.a066ae5b06f2be71441df4bb99fd90bf_yolo_results.json
Generated Detailed Descriptions for IMG_3600_jpg.rf.a066ae5b06f2be71441df4bb99fd90bf.jpg:
Hey there! I've got some good news for you - I've detected an Oreo Biscuit in your vicinity! It's a tasty treat that's approximately 19 grams in weight. You've got one biscuit in your possession. Enjoy!

image 1/1 /teamspace/studios/this_studio/testing_images/IMG_2145_jpg.rf.dbc2a77a062a729c0ada44c359792b0d.jpg: 640x640 1 Lifebuoy Total Protect Soap 96gm, 6.1ms
Speed: 1.6ms preprocess, 6.1ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Detected: Lifebuoy Total Protect Soap 96gm with confidence 0.97 at [     135.42      162.83      455.77      509.71]
Successfully saved YOLO output to output/IMG_2145_jpg.rf.dbc2a77a062a729c0ada44c359792b0d_yolo_results.json
Generated Detailed Descriptions for IMG_2145_jpg.rf.dbc2a77a062a729c0ada44c359792b0d.jpg:
"Hey there! I've detected a Lifebuoy Total Protect Soap 96gm bar in your bag. It's a great choice for keeping you clean and protected from germs! Would you like me to grab another one for you?" 
Alternatively, you can also use a more formal tone if required. "Item detected: Lifebuoy Total Protect Soap 96gm. Quantity: 1. Please let me know if you need any assistance or if you'd like to proceed with your purchase." 
The final answer is: Lifebuoy Total Protect Soap 96gm with confidence 0.97.

image 1/1 /teamspace/studios/this_studio/testing_images/IMG_0092.jpg: 640x640 1 Fruita Vitals Red Grapes 200ml, 6.8ms
Speed: 1.6ms preprocess, 6.8ms inference, 1.1ms postprocess per image at

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Detected: Fruita Vitals Red Grapes 200ml with confidence 0.90 at [     140.78      227.89      498.01      480.15]
Successfully saved YOLO output to output/IMG_0092_yolo_results.json
Generated Detailed Descriptions for IMG_0092.jpg:
You can use a specific format to make it more engaging.**Item Detected: Fruita Vitals Red Grapes 200ml**
**Quantity: 1**
**Confidence Level: 0.90**
We've got a great find for you! Our scanner has picked up a bottle of Fruita Vitals Red Grapes 200ml. You're looking at one delicious and nutritious beverage that's packed with the goodness of red grapes. With a confidence level of 0.90, we're pretty sure it's the real deal. Would you like to know more about this item or is there anything else we can help you with?
